# Fitting pure fluid model parameters

The cost function to be minimized is formed of a number of contributions:

* SatRhoLPoint: Saturated liquid density
* SatRhoLPPoint: Saturated liquid density and vapor pressure are combined into one contribution
* SatRhoLPWPoint: Saturated liquid density, vapor pressure, and speed of sound are combined into one contribution
* SOSPoint: Speed of sound for homogeneous state

Instances of these models are added to the ``PureParameterOptimizer`` instance via the ``add_one_contribution`` method.  The cost function is evaluated via either the ``cost_function`` method or the ``cost_function_threaded`` (the latter is threaded and you need to specify how many threads should be used as the second argument).

What optimization algorithm you want to use is up to you. I used differential evolution in the example here because it is available in scipy.optimize.

In [1]:
import teqp
import numpy as np
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import scipy.optimize
display(teqp.__version__)

'0.22.0'

In [2]:
nonpolar = {
    "kind": "SAFT-VR-Mie",
    "model": {
        "coeffs": [
        {
          "name": "R32",
          "BibTeXKey": "Bell",
          "m": 1.2476268271391935,
          "sigma_m": 3.6080717234117107e-10,
          "epsilon_over_k": 172.53065054286867,
          "lambda_r": 14.634722358167384,
          "lambda_a": 6
        }
      ]
    }
}

template = {  
  'kind': 'genericSAFT', 
  'model': {
      'nonpolar': nonpolar
  }
}

# These are the JSON pointers for the locations in the JSON where the fitted parameters should be inserted
# NOTE: '/' inside field names MUST be escaped as ~1; see https://datatracker.ietf.org/doc/html/rfc6901#section-3
pointers = [
    '/model/nonpolar/model/coeffs/0/m',
    '/model/nonpolar/model/coeffs/0/sigma_m',
    '/model/nonpolar/model/coeffs/0/epsilon_over_k',
    '/model/nonpolar/model/coeffs/0/lambda_r',
    '/model/nonpolar/model/coeffs/0/lambda_a'
]
x0 = [1.5, 3e-10, 150, 19, 5.7]
bounds = [(1,5),(2e-10,5e-10),(100,400),(8,25),(5.1, 7.0)]

In [3]:
FLD = 'Propane'
ppo = teqp.paramopt.PureParameterOptimizer(template, pointers)

Ts = np.linspace(230, 330)
for T in Ts:
    pt = teqp.paramopt.SatRhoLPPoint()
    rhoL, rhoV = [CP.PropsSI('Dmolar','Q',Q,'T',T,FLD) for Q in [0,1]]
    p = CP.PropsSI('P','Q',0,'T',T,FLD)
    pt.T = T
    # Measurands (here, pseudo-experimental values coming from the reference EOS)
    pt.p_exp = p
    pt.rhoL_exp = rhoL
    # Control parameters
    pt.rhoL_guess = rhoL
    pt.rhoV_guess = rhoV
    pt.weight_p = 100
    pt.weight_rho = 100
    ppo.add_one_contribution(pt)
    
def cost_function(x):
    return ppo.cost_function_threaded(x, 4)
print(cost_function(x0))

r = scipy.optimize.differential_evolution(cost_function, bounds=bounds, disp=True, maxiter=10000)
print(r)
x = r.x
model = teqp.make_model(ppo.build_JSON(x))

AttributeError: module 'teqp' has no attribute 'paramopt'

In [4]:
# Plot the rho-T data
Tc, rhoc = model.solve_pure_critical(400, 5000)
# print(Tc, rhoc)
anc = teqp.build_ancillaries(model, Tc, rhoc, 0.5*Tc)
RHOL, RHOV, PPP = [],[],[]
z = np.array([1.0])
Tsverify = np.linspace(230, Tc*0.9, 1000)
for T in Tsverify:
    rhoL, rhoV = model.pure_VLE_T(T, anc.rhoL(T), anc.rhoV(T), 10)
    p = rhoL*model.get_R(z)*T*(1+model.get_Ar01(T, rhoL, z))
    RHOL.append(rhoL)
    RHOV.append(rhoV)
    PPP.append(p)
    
# Plot deviation plots in the fitted data
line, = plt.plot(np.array(RHOL), Tsverify)
plt.plot(np.array(RHOV), Tsverify, color=line.get_color())
for Q in [0, 1]:
    D = CP.PropsSI('Dmolar','T',Tsverify,'Q',Q,FLD)
    plt.plot(D, Tsverify, lw=2, color='k')
plt.gca().set(xlabel=r'$\rho$ / mol/m$^3$', ylabel='$T$ / K')

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(10,7), sharex=True)

ax1.plot(Tsverify, (np.array(PPP)/CP.PropsSI('P','T',Tsverify,'Q',0,FLD)-1)*100)
ax1.set(ylabel=r'$(p_{fit}/p_{\rm pexp}-1)\times 100$')

ax2.plot(Tsverify, (np.array(RHOL)/CP.PropsSI('Dmolar','T',Tsverify,'Q',0,FLD)-1)*100, label='liquid')
ax2.plot(Tsverify, (np.array(RHOV)/CP.PropsSI('Dmolar','T',Tsverify,'Q',1,FLD)-1)*100, label='vapor')
ax2.legend()
ax2.set(ylabel=r'$(\rho_{fit}/\rho_{\rm pexp}-1)\times 100$', xlabel='$T$ / K');

NameError: name 'model' is not defined